In [1154]:
from typing import List

import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [1155]:
# Instantiate the data; you'll need to upload this file, or download the notebook and run it locally if you want this to work. 
from collections import Counter

from sklearn.preprocessing import MultiLabelBinarizer


data = pd.read_csv('./hw1_train.csv', index_col=0)
data.columns = ["text", "labels"]
                                                                                                                                                              
data["labels"].fillna("none", inplace=True)
data["labels"] = data["labels"].str.split()

vocab_size = 1_000  # How many tokens to include in the vocabulary. Feel free to adjust this!
        
# create self.labels: a list of every possible label in the dataset
labels = (
    data
    .drop(columns='text')
    .explode('labels')
    ['labels'].unique()
).tolist()
labels.remove('none')
n_labels = len(labels)
        
# create self.label2id: a dictionary which maps the labels from self.labels (above) to a unique integer
label2id = {v: k for k, v in dict(enumerate(labels)).items()}

mlb = MultiLabelBinarizer()
mlb.fit([labels])
print(mlb.classes_)

df = data.copy()
df["text"] = df["text"].str.split()
## TODO REMOVE APOSTROPHE S MAYBE
vocab = (
    df
    .drop(columns='labels')
    .explode('text')
)['text'].tolist()
    
vocab = Counter(vocab).most_common(vocab_size)
for i in range(vocab_size):
    y = list(vocab[i])
    y[1] = i
    vocab[i] = tuple(y)
    # also, don't forget to include <unk> (unknown)
    # TODO assign <unk> a unique ID. 
    # ======================================================================
    
vocab.append(('<unk>', vocab_size))
vocab_size = vocab_size + 1 # plus 1 because <unk>

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

['actor.gender' 'gr.amount' 'movie.country' 'movie.directed_by'
 'movie.estimated_budget' 'movie.genre' 'movie.gross_revenue'
 'movie.initial_release_date' 'movie.language' 'movie.locations'
 'movie.music' 'movie.produced_by' 'movie.production_companies'
 'movie.rating' 'movie.starring.actor' 'movie.starring.character'
 'movie.subjects' 'person.date_of_birth']


In [1156]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create the transform
kwargs = {
    'ngram_range': (1,1),  # Use 1-grams + 2-grams.
    'analyzer': 'word',  # Split text into word tokens.
    'min_df': 1,
    'stop_words': "english",
}
vectorizer = TfidfVectorizer(**kwargs)
vectorizer.fit(train_data['text'])

TfidfVectorizer(stop_words='english')

In [1157]:
from cmath import isnan
from collections import Counter
import scipy

from sklearn.preprocessing import MultiLabelBinarizer


class MovieDataset(Dataset):
    def __init__(self, df: pd.DataFrame, vocab):
        self.data = df
        self.vocab = vocab

    
    def one_hot_encode_labels(self, labels: List[str]):        
        # TODO CHANGE TO MULTILABELBINALIZER?        
        # encoded = np.zeros(n_labels)
        # for element in labels:
        #     if element in label2id:
        #         encoded[label2id.get(element)] = 1
        # encoded = encoded.astype(np.float32)
        # return torch.from_numpy(encoded)
        encoded = mlb.transform([labels])
        encoded = encoded.astype(np.float32)
        encoded = torch.squeeze(torch.from_numpy(encoded))
        return encoded
    
    def tokenize(self, text: str):
        return text.split()
    
    def encode_tokens(self, tokens: str):
        vector = vectorizer.transform([tokens])
        encoded = vector.toarray()
        encoded = encoded.astype(np.float32)
        encoded = torch.squeeze(torch.from_numpy(encoded))
        return encoded

    def __len__(self):
        return len(self.data)

    def __getitem__(self, n: int):
        input_to_model = self.data["text"].iloc[n]
        labels = self.data["labels"].iloc[n]
        return self.encode_tokens(input_to_model), self.one_hot_encode_labels(labels)



In [1158]:
#TODO tuple output, fix

train_set = MovieDataset(train_data, vocab)
eval_set = MovieDataset(val_data, vocab)

# A small batch size of 2 makes it easier to debug for printing. 
train_dataloader = DataLoader(train_set, batch_size=1)
eval_dataloader = DataLoader(eval_set, batch_size=1)

emsize = (len(train_dataloader.dataset[0][0]))
print(emsize)

906


In [1159]:
# Zipping the dataloader with range(N) lets us only print the first N batches
for _, batch in zip(range(5), train_dataloader):
    # Do something here; maybe print the batch to see if it looks right to you?
    print(batch[0].shape, batch[1].shape)

torch.Size([1, 906]) torch.Size([1, 18])
torch.Size([1, 906]) torch.Size([1, 18])
torch.Size([1, 906]) torch.Size([1, 18])
torch.Size([1, 906]) torch.Size([1, 18])
torch.Size([1, 906]) torch.Size([1, 18])


In [1162]:
# import torch.nn as nn
# import torch.nn.functional as F

# inputs  = 1016
# outputs = 18
 
# net = nn.Sequential(
#       nn.Linear(inputs, 508),
#       nn.ReLU(),
#       nn.Linear(508, outputs),
#       nn.Sigmoid()
#       ).to(device)


In [1163]:
# # define the model
# import torch.nn as nn
# import torch.nn.functional as F

# class MyModel(nn.Module):

#     def __init__(self, embed_dim, hidden_dim, output_dim):
#         super().__init__()
#         #self.embedding = nn.EmbeddingBag(n_tokens, embed_dim, sparse=True)
#         # self.init_weights()
#         self.relu1 = nn.ReLU()
#         self.relu2 = nn.ReLU()
#         self.relu3 = nn.ReLU()
#         self.fc1 = nn.Linear(int(embed_dim), int(hidden_dim))
#         self.fc2 = nn.Linear(int(hidden_dim), int(hidden_dim/2))
#         self.fc3 = nn.Linear(int(hidden_dim/2), int(output_dim))


#     # def init_weights(self):
#     #     initrange = 0.5
#     #     self.embedding.weight.data.uniform_(-initrange, initrange)
#     #     self.fc.weight.data.uniform_(-initrange, initrange)
#     #     self.fc.bias.data.zero_()

#     def forward(self, text):
#         # embedded = self.embedding(text.long())
#         hidden1 = self.relu1(self.fc1(text))
#         hidden2 = self.relu2(self.fc2(hidden1))
#         hidden3 = self.relu3(self.fc3(hidden2))
#         return hidden3

In [1169]:
# define the model
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
  
 
   def __init__(self, input_size, hidden_sizes, output_size, dropout=False, dropout_p=0.1):
       super(MyModel, self).__init__()
       self.fc1 = nn.Linear(input_size, hidden_sizes[0])
       self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
       self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
       self.fc4 = nn.Linear(hidden_sizes[2], output_size)
       # self.fc5 = nn.Sigmoid()
 
       self.add_dropout = dropout
       self.dropout = nn.Dropout(dropout_p)
 
   def forward(self, x):
       h1 = F.relu(self.fc1(x))
       h2 = F.relu(self.fc2(h1))
       h3 = F.relu(self.fc3(h2))
       # h4 = F.relu(self.fc4(h3))
       if self.add_dropout:
           logits = self.fc4(self.dropout(h3))
       else:
           logits = self.fc4(h3)
       return logits


In [1170]:

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    
n_epochs = 10
learning_rate = 1e-3
hsize = emsize/2
hidden_sizes = [500, 200, 64]
output_size = n_labels

model = MyModel(emsize, hidden_sizes, output_size).to(device)

In [1171]:
from torch import logit
from sklearn.metrics import f1_score

def train(loader, model, optimizer, loss_fn):
    model.train()
    losses = []
    pbar = tqdm(loader)
    for x, y in pbar:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()

        logits = model(x)
        loss = loss_fn(logits, y)
        pbar.set_postfix({'loss': loss.item()})
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
    return sum(losses) / len(losses)


def evaluate(loader, model, loss_fn, score_fn):
    model.eval()
    predictions = []
    lbs = []
    for x, y in tqdm(loader):
        x = x.to(device)
        y = y.to(device)

        logits = model(x)
        # loss = loss_fn(logits, y)

        pred = torch.sigmoid(logits)
        pred = pred.detach()
        pred = np.round(np.array(pred.cpu()))

        predictions.append(pred)
        lbs.append(y.cpu().numpy())
    
    predictions =  np.array(predictions)
    lbs = np.array(lbs)
    predictions = np.reshape(predictions, (predictions.shape[0], 18))
    lbs = np.reshape(lbs, (lbs.shape[0], 18))


    score = score_fn(lbs, predictions, target_names=labels)
    f1score = f1_score(lbs, predictions, average="weighted")
    return f1score
    

In [1172]:
from sklearn.metrics import classification_report

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = torch.nn.BCEWithLogitsLoss()
score_fn = classification_report
best_acc = 0
# with open("output.txt", "a") as f:
#     for epoch in range(n_epochs):
#         avg_loss = train(train_dataloader, model, optimizer, loss_fn)
#         accuracy = evaluate(eval_dataloader, model, loss_fn, score_fn)
#         print('train loss: ', avg_loss, file=f)
#         print('val accuracy: \n ', accuracy, file=f)
#     # if accuracy > best_acc and accuracy > 0.7:
#     #     torch.save(model.state_dict(), f'best-model.pt')

In [1173]:
from numpy import split
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

foldperf = {}
k = 10
splits = KFold(n_splits = k, shuffle = True, random_state = 42)
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(data)))):

    print('Fold {}'.format(fold + 1))
    history = {'train_loss': [], 'eval_score':[]}

    for epoch in range(1):
        train_loss = train(train_dataloader, model, optimizer, loss_fn)
        eval_score = evaluate(eval_dataloader, model, loss_fn, score_fn)
        print(train_loss)
        print(eval_score)

        print("AVG Training Loss:{:.3f} % AVG Test Acc {:.2f} %".format(train_loss, eval_score))
        print("---------------------------------------------\n")
        history['train_loss'].append(train_loss)
        history['eval_score'].append(eval_score)

    foldperf['fold{}'.format(fold+1)] = history  

torch.save(model,'myModel.pt')    

Fold 1


100%|██████████| 463/463 [00:00<00:00, 845.95it/s]


0.16277670935733826
0.512199154040608
AVG Training Loss:0.163 % AVG Test Acc 0.51 %
---------------------------------------------

Fold 2


100%|██████████| 463/463 [00:00<00:00, 820.67it/s]


0.0660432435903973
0.7656241268764131
AVG Training Loss:0.066 % AVG Test Acc 0.77 %
---------------------------------------------

Fold 3


...

0.010227203276237835
0.8255788562517296
AVG Training Loss:0.010 % AVG Test Acc 0.83 %
---------------------------------------------

Fold 9


100%|██████████| 463/463 [00:00<00:00, 821.73it/s]


0.008200756416875817
0.8303300495329
AVG Training Loss:0.008 % AVG Test Acc 0.83 %
---------------------------------------------

Fold 10


100%|██████████| 463/463 [00:00<00:00, 837.07it/s]

0.007404133098130679
0.8275412783701537
AVG Training Loss:0.007 % AVG Test Acc 0.83 %
---------------------------------------------



In [1191]:
model = torch.load('myModel.pt')

test_data = pd.read_csv('./hw1_test.csv')

x_submission = np.array(test_data['UTTERANCES'])
x_submission_transformed = vectorizer.transform(x_submission).toarray()
x_submission_transformed = x_submission_transformed.astype(np.float32)
x_submission_transformed_ts = torch.Tensor(x_submission_transformed)

def prediction_func(x_submission, model):
    predictions = []
    for i , x in enumerate(x_submission):
        logits = model(x)
        pred = torch.sigmoid(logits)
        pred = pred.detach()
        pred = np.round(np.array(pred.cpu()))
        predictions.append(pred)
    predictions = np.array(predictions)
    pred_classes = mlb.inverse_transform(predictions)
 
    reformat_pred = []
    for pred in pred_classes:
       pred_lst = []
       for i in range(len(pred)):
         pred_lst.append(pred[i])
       reformat_pred.append(np.array(pred_lst))
    return np.array(reformat_pred)


In [1193]:
preds = prediction_func(x_submission_transformed_ts, model)

test_data['CORE RELATIONS'] = preds

test_data['CORE RELATIONS'] = test_data['CORE RELATIONS'].apply(lambda x: " ".join(x))

submission_data = test_data[['ID', 'CORE RELATIONS']]

submission_data.to_csv('submission.csv', index = False)
